# Regression Model A: Field 4

## Install libraries

In [1]:
install.packages('readxl')
install.packages('corrplot')
install.packages('mice')
install.packages('RColorBrewer')
install.packages('car')
install.packages('Metrics')

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("readxl"):
“installation of package ‘readxl’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("mice"):
“installation of package ‘mice’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


## Call libraries

In [2]:
library(readxl)
library(corrplot)
library(mice)
library(RColorBrewer)
library(car)
library(Metrics)

corrplot 0.84 loaded
Loading required package: lattice
Loading required package: carData


## Import data

In [3]:
d1=read_excel("variable2.xlsx",sheet = 1)
d2=read_excel("variable2.xlsx",sheet = 2)
d3=read_excel("variable2.xlsx",sheet = 3) 
d4=read_excel("variable2.xlsx",sheet = 4)
d5=read_excel("variable2.xlsx",sheet = 5) 

## Field 4 (F4)
Call the data according to the Field. 

In [4]:
dE1=d1[which(d1$`ESTATE CODE`=="E4" ),]
dE2=d2[(d2$Diff ),]
dE3=d3[which(d3$Estate=="E4" ),]
dE4=d4[which(d4$Estate=="E4" ),]
dE5=d5[which(d5$Estate=="E4" ),]

## Variables

Declare the variable and construct new table for easy representation. 

In [5]:
year=dE1$`CALENDER YEAR`
month=dE1$`CALENDER MONTH`
yield=dE1$`CPB TONNAGE`
rainfall=dE1$RAINFALL
foliar=d2$Diff
prun.time=dE3$Frequency
prun.cost=dE3$PruningCost
manuring.time=dE4$Frequency
manuring.cost=dE4$ManuringCost
pd.time=dE5$Frequency
pd.cost=dE5$PDCost

dg1=data.frame(year,month,yield,rainfall,foliar,prun.time,prun.cost,
               manuring.time,manuring.cost,pd.time,pd.cost)

## Correlation

Check the correlations for all variables. 

In [6]:
c1=cor(dg1[,-1:-2])

corrplot(c1, method="number",type="upper",col=brewer.pal(n=8, name="RdYlBu"))
corrplot(c1, type="upper")

## Train and test data

Split the dataset into train and test sets. 

After a model has been processed by using the training set, we test the model by making predictions against the test set. 

Because the data in the testing set already contains known values for the attribute that we want to predict, it is easy to determine whether the model's guesses are correct.

We use the datasets from May 2014 until December 2016 is considered for training dataset. The test dataset is from January 2017 until December 2017. 

In [7]:
train = dg1[dg1$year<=2016,]
test = dg1[dg1$year==2017,]
dim(train)
dim(test)

[1] 32 11

[1] 12 11

## Regression

Fit the data using multiple regression for all variables. 

In [8]:
a1=lm(yield~rainfall+foliar+prun.time+prun.cost+
        manuring.time+manuring.cost+pd.time+pd.cost,data=train)
summary(a1)
plot(a1,which=1)
plot(a1,which=2)


Call:
lm(formula = yield ~ rainfall + foliar + prun.time + prun.cost + 
    manuring.time + manuring.cost + pd.time + pd.cost, data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-170.41  -49.75   13.92   44.28  131.09 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)    3.483e+02  8.274e+01   4.210 0.000334 ***
rainfall       1.466e-02  4.294e-02   0.341 0.735856    
foliar        -9.488e+00  4.920e+00  -1.928 0.066228 .  
prun.time      1.628e+01  1.460e+01   1.115 0.276346    
prun.cost     -9.499e-03  1.019e-02  -0.933 0.360750    
manuring.time -5.682e+01  3.492e+01  -1.627 0.117407    
manuring.cost -2.167e-04  1.377e-03  -0.157 0.876328    
pd.time        2.035e+01  2.575e+01   0.790 0.437453    
pd.cost       -2.812e-03  6.156e-03  -0.457 0.652072    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 83.39 on 23 degrees of freedom
Multiple R-squared:  0.4512,	Adjusted R-squared:  0.260

## Significant variable 

Check the significant variables using two ways. 

a.	anova  
Choose the variables that has significant value in which p<0.1.

b.	using stepwise method  
In order to avoid the multicollinearity issues in variables selection, we take into account the variance inflation factors (VIF). We remove the variables that has VIF greater than 4. Then we rebuild the model. 


The stepwise method incorporate with VIF factor is better than anova method in order to     select the significant variables.    

### Using ANOVA

In [9]:
anova(a1)

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
rainfall,1,710.18722,710.18722,0.102119936,0.752184876
foliar,1,72626.55016,72626.55016,10.443187952,0.003689054
prun.time,1,1701.04202,1701.04202,0.244597899,0.625595909
prun.cost,1,15664.61804,15664.61804,2.252462083,0.147006197
manuring.time,1,36349.13204,36349.13204,5.226749955,0.031779766
manuring.cost,1,62.87911,62.87911,0.009041575,0.925069160
pd.time,1,2914.00298,2914.00298,0.419013168,0.523838731
pd.cost,1,1451.40513,1451.40513,0.208701866,0.652072131
Residuals,23,159952.17757,6954.44250,NA,NA


### Using stepwise

In [10]:
selectedMod <- step(a1)
summary(selectedMod)

##Avoid Multicollinearity and Statistical Significance
##Recursively remove variables with variance inflation factors: VIF > 4
# Remove vars with VIF> 4 and re-build model until none of VIFs don't exceed 4.
all_vifs <- car::vif(selectedMod)
print(all_vifs)

while(any(all_vifs > 4)){
  var_with_max_vif <- names(which(all_vifs == max(all_vifs)))  # get the var with max vif
  signif_all <- signif_all[!(signif_all) %in% var_with_max_vif]  # remove
  myForm <- as.formula(paste("ozone_reading ~ ", paste (signif_all, collapse=" + "), sep=""))  # new formula
  selectedMod <- lm(myForm, data=inputData)  # re-build model with new formula
  all_vifs <- car::vif(selectedMod)
}

summary(selectedMod)

Start:  AIC=290.54
yield ~ rainfall + foliar + prun.time + prun.cost + manuring.time + 
    manuring.cost + pd.time + pd.cost

                Df Sum of Sq    RSS    AIC
- manuring.cost  1     172.2 160124 288.57
- rainfall       1     810.9 160763 288.70
- pd.cost        1    1451.4 161404 288.83
- pd.time        1    4343.0 164295 289.40
- prun.cost      1    6047.6 166000 289.73
- prun.time      1    8646.7 168599 290.23
<none>                       159952 290.54
- manuring.time  1   18404.4 178357 292.03
- foliar         1   25863.9 185816 293.34

Step:  AIC=288.58
yield ~ rainfall + foliar + prun.time + prun.cost + manuring.time + 
    pd.time + pd.cost

                Df Sum of Sq    RSS    AIC
- rainfall       1       870 160994 286.75
- pd.cost        1      1412 161536 286.86
- pd.time        1      4234 164358 287.41
- prun.cost      1      8789 168914 288.29
- prun.time      1     10258 170383 288.56
<none>                       160124 288.57
- foliar         1     26043 18


Call:
lm(formula = yield ~ foliar + prun.time + prun.cost + manuring.time, 
    data = train)

Residuals:
     Min       1Q   Median       3Q      Max 
-151.721  -57.791    4.552   42.109  136.659 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)   405.037963  29.810735  13.587 1.38e-13 ***
foliar         -9.562915   4.476937  -2.136   0.0419 *  
prun.time      19.806425  11.758245   1.684   0.1036    
prun.cost      -0.011464   0.007802  -1.469   0.1533    
manuring.time -62.626593  25.730862  -2.434   0.0218 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 78.09 on 27 degrees of freedom
Multiple R-squared:  0.435,	Adjusted R-squared:  0.3513 
F-statistic: 5.197 on 4 and 27 DF,  p-value: 0.003087


       foliar     prun.time     prun.cost manuring.time 
     1.156921      2.029118      2.114964      1.136587 



Call:
lm(formula = yield ~ foliar + prun.time + prun.cost + manuring.time, 
    data = train)

Residuals:
     Min       1Q   Median       3Q      Max 
-151.721  -57.791    4.552   42.109  136.659 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)   405.037963  29.810735  13.587 1.38e-13 ***
foliar         -9.562915   4.476937  -2.136   0.0419 *  
prun.time      19.806425  11.758245   1.684   0.1036    
prun.cost      -0.011464   0.007802  -1.469   0.1533    
manuring.time -62.626593  25.730862  -2.434   0.0218 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 78.09 on 27 degrees of freedom
Multiple R-squared:  0.435,	Adjusted R-squared:  0.3513 
F-statistic: 5.197 on 4 and 27 DF,  p-value: 0.003087


## Prediction

The performance of the model is evaluated by comparing the estimated value with the actual value. In this case, the test dataset (2017 datasets) is used to examine the performance of the model with significant variables. 

Construct the table and graph that represents the estimated and observed value. 

In [11]:
estimate=predict(selectedMod,newdata=test)
compare=data.frame(test$yield,estimate)
names(compare)=c("Actual","Estimate")
compare

plot(compare$Actual,type="b",col="blue",xlab="Actual",ylab="Predicted",pch=19)
lines(fitted(selectedMod),col="red",type="b",pch=18)
legend("topleft",legend=c("Actual","Predicted"),col=c("blue","red"),lty=1,cex=0.8)

,Actual,Estimate
,<dbl>,<dbl>
33,347.478,328.5346
34,188.128,193.7185
35,486.897,246.7822
36,517.121,104.8427
37,469.213,367.4512
38,402.958,453.9329
39,413.973,377.8215
40,373.536,301.5571
41,298.210,367.3233


## Root mean square error (RMSE)

Calculate the root mean square error (RMSE) of the model. Extract the coefficient of determination (R-square) to know how close the data to the fitted regression line. 

In [12]:
rmse=rmse(test$yield,estimate )
fit=c(RMSE = rmse, R2=summary(a1)$r.squared)
fit

RMSE          R2 
146.6811731   0.4511509